In [ ]:

#Estas son todas las librerías que vamos a utilizar durante la práctica.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor




In [ ]:
# Lectura de los dos CSV:

df_examples = pd.read_csv('uber_time_examples (2).csv')
df_labels = pd.read_csv('uber_time_labels (2).csv')


In [ ]:
#Unimos los data frames en base a la columna id:

df_combined = pd.merge(df_examples, df_labels, on='id')


#Reducimos de 400 000 a 20 000 el número de registros
#para poder ejecutar el entrenamiento de models de forma más rápida.

df_combined = df_combined.sample(n=20000, random_state=42)

#Formateo de la columna de fecha y hora del viaje y eliminación
#de valores nulos:

# Agregar el año a los valores de 'feature_0' y convertir a datetime
df_combined['feature_0'] = '2024-' + df_combined['feature_0']
df_combined['feature_0'] = pd.to_datetime(df_combined['feature_0'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Eliminar filas con valores nulos en 'feature_0'
df_combined = df_combined.dropna(subset=['feature_0'])





 A partir de la columna de fecha y hora, se puede obtener la hora
 en la que se produjo el viaje como una atributo independiente y
 el día de la semana, lo cuál puede ser de utilidad para entrenar el
 modelo.

 Así se puede sacar el día de la semana y la hora de recogida como
 atributo independiente.


In [ ]:


df_combined['pickup_hour'] = df_combined['feature_0'].dt.hour
df_combined['pickup_day_of_week'] = df_combined['feature_0'].dt.dayofweek

# Eliminación de valores nulos para prevenir errores:

df_combined = df_combined.dropna(subset=['duration'])




Es necesario eliminar todos los valores cercanos a 0 o a infinito, para no distorsionar el entrenamiento del modelo.

Esto lo hacemos en el siguiente paso.


In [ ]:


q_low = df_combined['duration'].quantile(0.01)
q_high = df_combined['duration'].quantile(0.99)
df_combined = df_combined[(df_combined['duration'] > q_low) & (df_combined['duration'] < q_high)]


In [ ]:
# Los siguientes pasos van dirigidos a gestionar valores faltantes y nulos,
# así como transformar y modificar las variables que sean necesarias
# para que el model las pueda interpretar correctamente.


# Separar características numéricas y categóricas:
num_features = df_combined.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = df_combined.select_dtypes(include=['object']).columns.tolist()
num_features.remove('duration')

# Definir transformadores para características numéricas y categóricas.
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('power_transformer', PowerTransformer(method='yeo-johnson'))
])



In [ ]:
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Crear un preprocesador que aplica las transformaciones a las variables
#donde es necesario.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ]
)

Dividir el conjunto de datos en train y test:

El conjunto de entrenamiento se usará para entrenar el modelo
y el de prueba para verificar su exactitud (que se mide con el MAPE).

In [ ]:


X = df_combined.drop('duration', axis=1)
y = df_combined['duration']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y ajustar el preprocesador:
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Selección de las características más relevantes:
selector = SelectKBest(score_func=f_regression, k=10)
X_train_selected = selector.fit_transform(X_train_preprocessed, y_train)
X_test_selected = selector.transform(X_test_preprocessed)

# Modelo sencillo: Linear regression:
model_lr = LinearRegression()
model_lr.fit(X_train_selected, y_train)
y_pred_lr = model_lr.predict(X_test_selected)
mape_lr = mean_absolute_percentage_error(y_test, y_pred_lr)
print(f'mean_abs_perc_error del model Linear Regression: {mape_lr:.2f}')



MAPE del modelo Linear Regression: 0.33


**En este caso, el MAPE obtenido es del 33% para el modelo de regresión lineal**

Este resultado indica que el modelo de regresión lineal predice con un error aproximado del 33%, lo cual señala una necesidad de mejorar su configuración o considerar otros modelos.


Para mejorar este valor, se pueden añadir más características
relevantes u ajustar los configuraciones (esto último lo vamos a hacer
en los dos siguientes modelos)

## 1: Random forest regressor:

 El modelo de random forest se basa en un conjunto de
 árboles de decisión combinados entre sí, de forma que
 los árboles se fortalezcan entre ellos aumentando la
 exactitud.

In [ ]:


model_rf = RandomForestRegressor(random_state=42)
model_rf.fit(X_train_selected, y_train)
y_pred_rf = model_rf.predict(X_test_selected)
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf)
print(f'mean_abs_perc_error del model Random Forest: {mape_rf:.2f}')

# En este caso, el mean_abs_perc_error obtenido es del 35%. En el siguiente
# apartado vamos a ver si optimizando los hiperparámetros conseguimos reducir
# este valor (lo que quiere decir que la precisión ha aumentado).

MAPE del modelo Random Forest: 0.35


### Optimización de hiper-parámetros para el modelo Random forest:

Se ajustan valores como el número de árboles en
el bosque, la profundida máxima de cada árbol y el número de muestras mínimo
para dividir un nodo. Estos son los parámetros a ajustar.

Se probarán varios configuraciones, y se escogerá el modelo que
minimiza el MAPE.


In [ ]:

param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
grid_search_rf = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                           param_grid=param_grid_rf,
                           scoring='neg_mean_absolute_percentage_error',
                           cv=3)
grid_search_rf.fit(X_train_selected, y_train)
best_rf_model = grid_search_rf.best_estimator_

y_pred_best_rf = best_rf_model.predict(X_test_selected)
mape_best_rf = mean_absolute_percentage_error(y_test, y_pred_best_rf)
print(f'mean_abs_perc_error del mejor model Random Forest: {mape_best_rf:.2f}')


#La optimización reduce el mean_abs_perc_error en un 2 %, bajando al 33% de error porcentual.


MAPE del mejor modelo Random Forest: 0.33


## 2: Red neuronal con Keras:

El segundo modelo avanzado que vamos a utilizar es el modelo de red neuronal con KERAS.


In [ ]:

def create_model(neurons=64, optimizer='adam'):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train_selected.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mean_absolute_percentage_error')
    return model

model_keras = KerasRegressor(model=create_model, verbose=0)

# Definir la cuadrícula de hiper-parámetros para la red neuronal
param_grid_keras = {
    'model__neurons': [32, 64, 128],
    'model__optimizer': ['adam', 'rmsprop'],
    'fit__batch_size': [32, 64],
    'fit__epochs': [20, 50]
}

# 2.1 Optimización de hiper-parámetros para el model de Keras:


# Después de la optimización, se selecciona el model con el mean_abs_perc_error más bajo.

grid_search_keras = GridSearchCV(estimator=model_keras,
                                 param_grid=param_grid_keras,
                                 scoring='neg_mean_absolute_percentage_error',
                                 cv=3)
grid_search_keras.fit(X_train_selected, y_train)
best_keras_model = grid_search_keras.best_estimator_

y_pred_best_keras = best_keras_model.predict(X_test_selected)
mape_best_keras = mean_absolute_percentage_error(y_test, y_pred_best_keras)
print(f'mean_abs_perc_error del mejor model de Red Neuronal: {mape_best_keras:.2f}')

#El mean_abs_perc_error después de optimizar este model sale del 29%, lo que indica
# que este model es el más preciso.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo

MAPE del mejor modelo de Red Neuronal: 0.29


### Conclusiones:


Mediante el ajuste meticuloso de las configuraciones de los modelos, hemos logrado una disminución significativa en el valor del error porcentual absoluto medio (MAPE) para cada modelo evaluado.

Sin embargo, la notable variabilidad en la duración de los viajes, reflejada en el atributo duration, ha limitado nuestra capacidad para reducir el MAPE por debajo del 29%., Esto sugiere que la incorporación de atributos adicionales al conjunto de datos original podría ser beneficiosa, potencialmente aumentando la exactitud del modelo al capturar más aspectos relevantes del fenómeno estudiado.

Tras la optimización de configuraciones, el análisis indica que la red neuronal implementada mediante KERAS sobresale como el modelo más exacto, seguido por el modelo de bosque aleatorio (Random Forest) y, en última instancia, la regresión lineal.